# Fred

In [2]:
import json
from aries_askar import Key, KeyAlg
from didcomm_messaging.crypto.backend.askar import AskarCryptoService, AskarSecretKey
from didcomm_messaging.crypto.backend.basic import InMemorySecretsManager
from didcomm_messaging.packaging import PackagingService
from didcomm_messaging.multiformats import multibase
from didcomm_messaging.multiformats import multicodec
from didcomm_messaging.resolver.peer import Peer2, Peer4
from didcomm_messaging.resolver import PrefixResolver
from did_peer_2 import KeySpec, generate, json
from didcomm_messaging import DIDCommMessaging
from didcomm_messaging.routing import RoutingService
from pydid.did import DID


# Define DIDComm Messaging Service Endpoint

In [3]:
didcomm_host = "localhost"
didcomm_port = 8766
didcomm_messaging_uri = f"ws://{didcomm_host}:{didcomm_port}"

# Create DID

In [4]:
secrets = InMemorySecretsManager()
crypto = AskarCryptoService()
verkey = Key.generate(KeyAlg.ED25519)
xkey = Key.generate(KeyAlg.X25519)
did = generate(
    [
        KeySpec.verification(
            multibase.encode(
                multicodec.wrap("ed25519-pub", verkey.get_public_bytes()),
                "base58btc",
            )
        ),
        KeySpec.key_agreement(
            multibase.encode(
                multicodec.wrap("x25519-pub", xkey.get_public_bytes()), "base58btc"
            )
        ),
    ],
    [
                {
                    "type": "DIDCommMessaging",
                    "serviceEndpoint": {
                        "uri": didcomm_messaging_uri,
                        "accept": ["didcomm/v2"],
                        "routingKeys": [],
                    },
                },
    ],
)
await secrets.add_secret(AskarSecretKey(verkey, f"{did}#key-1"))
await secrets.add_secret(AskarSecretKey(xkey, f"{did}#key-2"))


In [5]:
secrets

In [6]:
resolver = PrefixResolver({"did:peer:2": Peer2(), "did:peer:4": Peer4()})

did_document = await resolver.resolve(did)
print(json.dumps(did_document, indent=2))

{
  "@context": [
    "https://www.w3.org/ns/did/v1",
    "https://w3id.org/security/multikey/v1"
  ],
  "id": "did:peer:2.Vz6Mkskipyoo2GKrzaHhCLvJFT6YcUQqL5g37YTMjmXHLXWuw.Ez6LSnEHXZ79YBBdcYtC9xEFj5uR2Jhej1323XqpNJwqnJ7b3.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjYiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",
  "verificationMethod": [
    {
      "type": "Multikey",
      "id": "#key-1",
      "controller": "did:peer:2.Vz6Mkskipyoo2GKrzaHhCLvJFT6YcUQqL5g37YTMjmXHLXWuw.Ez6LSnEHXZ79YBBdcYtC9xEFj5uR2Jhej1323XqpNJwqnJ7b3.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjYiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",
      "publicKeyMultibase": "z6Mkskipyoo2GKrzaHhCLvJFT6YcUQqL5g37YTMjmXHLXWuw"
    },
    {
      "type": "Multikey",
      "id": "#key-2",
      "controller": "did:peer:2.Vz6Mkskipyoo2GKrzaHhCLvJFT6YcUQqL5g37YTMjmXHLXWuw.Ez6LSnEHXZ79YBBdcYtC9xEFj5uR2Jhej1323XqpNJwqnJ7b3.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjYiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",

# Instantiate DIDComm Messaging 

This is used to pack and unpack DIDComm messages.

In [7]:
packaging = PackagingService()
router = RoutingService()

dmp = DIDCommMessaging(
    crypto=crypto,
    secrets=secrets,
    resolver=resolver,
    packaging=packaging,
    routing=router,
)

# Create send message functon

Sends packed DIDComm messages to a websocket endpoint

In [8]:
async def send_message(message, to, frm):
    packy = await dmp.pack(
        message=message,
        to=to,
        frm=frm,
    )
    packed = packy.message            
    # Get the first http endpoint from the last DID in the DID chain
    endpoint = packy.get_endpoint("ws")
    async with websockets.connect(endpoint) as websocket:
        message = "Hello from the client!"
        print(f"Sending message: {packed}")
        await websocket.send(packed)  # Send message to the server
        
        response = await websocket.recv()  # Wait for the server's response
        print(f"Received response: {response}")

# Setup DIDComm Messaging Websocket

This is where DIDComm messages are sent to the coordinator

In [9]:
import asyncio
import websockets

inbox = []


# This handler will be called for every client connection.
async def handle_messages(websocket):
    print("Client connected")
    try:
        async for message in websocket:
            print(f"Received: {message}")
            # Echo the received message back to the client.
            # await websocket.send(f"Echo: {message}")

            unpacked =  await dmp.packaging.unpack(crypto, resolver, secrets,message)
            msg = json.loads(unpacked[0].decode())
            inbox.append(msg)
            print(f"Unpacked Message : {unpacked}")
    except websockets.exceptions.ConnectionClosed as e:
        print("Connection closed", e)

# The main function to start the server.
async def run_websocket():
    # Start the server on localhost at port 8765.
    async with websockets.serve(handle_messages, didcomm_host, didcomm_port):
        print(f"WebSocket server started on {didcomm_messaging_uri}")
        # Run the server until it is manually stopped.
        await asyncio.Future()  # Run forever

In [10]:
asyncio.create_task(run_websocket())

<Task pending name='Task-7' coro=<run_websocket() running at /tmp/ipykernel_9483/2641164070.py:24>>

WebSocket server started on ws://localhost:8766
Client connected
Received: b'{"protected": "eyJ0eXAiOiAiYXBwbGljYXRpb24vZGlkY29tbStlbmNyeXB0ZWQiLCAiYWxnIjogIkVDREgtMVBVK0EyNTZLVyIsICJlbmMiOiAiQTI1NkNCQy1IUzUxMiIsICJhcHUiOiAiWkdsa09uQmxaWEk2TWk1V2VqWk5hM04yVkhCak5YcGlWM0ptWkZwRGJ6Tk9lV3BCUWxGUlZVUnhRemQ1YTNONVFtczNVRmx0UTNGUk56UnBMa1Y2Tmt4VFoxSk5XVWhPUkhaeGNqa3pXR2xDY0RWeFRuUkdNWFI2VVRGamJUTm5TMkZ2WlZsMWEyTnlia1ZDTVhBdVUyVjVTakJKYW05cFdrY3dhVXhEU25wSmFuQTNTVzVXZVdGVFNUWkpibVI2VDJrNGRtSkhPV3BaVjNodllqTk9NRTlxWnpOT2FsVnBURU5LYUVscWNHSkpiVkp3V2tkT2RtSlhNSFprYWtscFdGTjNhV05wU1RaWE1URTVabEVqYTJWNUxUSSIsICJhcHYiOiAiUTZSTG1vOHVrNHV0V3lkZXpIdnJrdWo4OUZPbDZIbDJNUjBka1ZoTHN0RSIsICJlcGsiOiB7ImNydiI6ICJYMjU1MTkiLCAia3R5IjogIk9LUCIsICJ4IjogImlDaExqSWQ2cU5hZC12eGtXdTJYQzFtZXBNR0d5Ty00N2RBWm5pM081SDAifSwgInNraWQiOiAiZGlkOnBlZXI6Mi5WejZNa3N2VHBjNXpiV3JmZFpDbzNOeWpBQlFRVURxQzd5a3N5Qms3UFltQ3FRNzRpLkV6NkxTZ1JNWUhORHZxcjkzWGlCcDVxTnRGMXR6UTFjbTNnS2FvZVl1a2NybkVCMXAuU2V5SjBJam9pWkcwaUxDSnpJanA3SW5WeWFTSTZJ

# Get Coordinator DID

How this happens is out of band. 

In [11]:
coordinator_did='did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ'


# Subscribe to MuSig2 Coordinator

In [12]:
import uuid
message = {
        "type": "https://didcomm.org/musig2/subscribe",
        "id": str(uuid.uuid4()),
        "body": {},
        "from": did,
        "to": coordinator_did,
}

In [13]:
packy = await dmp.pack(
    message=message,
    to=coordinator_did,
    frm=did,
)
packed = packy.message    

In [14]:
json.loads(packed)["recipients"]

[{'encrypted_key': 'HbMC8Y01xiedCinz5aovWCwvOHqg1AYkjvzhcCYdcsaWV34SscHQiG6VISnxfuaQFvpNa-KrSp4xtb_4ctfYREpmAsz6fCSe',
  'header': {'kid': 'did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ#key-2'}}]

In [15]:
json.loads(packed)

{'protected': 'eyJ0eXAiOiAiYXBwbGljYXRpb24vZGlkY29tbStlbmNyeXB0ZWQiLCAiYWxnIjogIkVDREgtMVBVK0EyNTZLVyIsICJlbmMiOiAiQTI1NkNCQy1IUzUxMiIsICJhcHUiOiAiWkdsa09uQmxaWEk2TWk1V2VqWk5hM05yYVhCNWIyOHlSMHR5ZW1GSWFFTk1ka3BHVkRaWlkxVlJjVXcxWnpNM1dWUk5hbTFZU0V4WVYzVjNMa1Y2Tmt4VGJrVklXRm8zT1ZsQ1FtUmpXWFJET1hoRlJtbzFkVkl5U21obGFqRXpNak5ZY1hCT1NuZHhia28zWWpNdVUyVjVTakJKYW05cFdrY3dhVXhEU25wSmFuQTNTVzVXZVdGVFNUWkpibVI2VDJrNGRtSkhPV3BaVjNodllqTk9NRTlxWnpOT2FsbHBURU5LYUVscWNHSkpiVkp3V2tkT2RtSlhNSFprYWtscFdGTjNhV05wU1RaWE1URTVabEVqYTJWNUxUSSIsICJhcHYiOiAiN1B2cUNRS0pvbDQwN3pwLW5jeHBKSTZvZmNoQkxBeGJFVVhYSl9COHJEdyIsICJlcGsiOiB7ImNydiI6ICJYMjU1MTkiLCAia3R5IjogIk9LUCIsICJ4IjogIktYUEFEZFZzNWs1LXNtUFNscHFkT2N1M0FKTzFxanN3eE5OUTNYdUc2aFkifSwgInNraWQiOiAiZGlkOnBlZXI6Mi5WejZNa3NraXB5b28yR0tyemFIaENMdkpGVDZZY1VRcUw1ZzM3WVRNam1YSExYV3V3LkV6NkxTbkVIWFo3OVlCQmRjWXRDOXhFRmo1dVIySmhlajEzMjNYcXBOSndxbko3YjMuU2V5SjBJam9pWkcwaUxDSnpJanA3SW5WeWFTSTZJbmR6T2k4dmJHOWpZV3hvYjNOME9qZzNOallpTENKaElqcGJJbVJwWkdOdmJXMHZkaklpWFN3aWNpS

In [16]:
asyncio.create_task(send_message(message, coordinator_did, did))

<Task pending name='Task-10' coro=<send_message() running at /tmp/ipykernel_9483/1218273840.py:1>>

Sending message: b'{"protected": "eyJ0eXAiOiAiYXBwbGljYXRpb24vZGlkY29tbStlbmNyeXB0ZWQiLCAiYWxnIjogIkVDREgtMVBVK0EyNTZLVyIsICJlbmMiOiAiQTI1NkNCQy1IUzUxMiIsICJhcHUiOiAiWkdsa09uQmxaWEk2TWk1V2VqWk5hM05yYVhCNWIyOHlSMHR5ZW1GSWFFTk1ka3BHVkRaWlkxVlJjVXcxWnpNM1dWUk5hbTFZU0V4WVYzVjNMa1Y2Tmt4VGJrVklXRm8zT1ZsQ1FtUmpXWFJET1hoRlJtbzFkVkl5U21obGFqRXpNak5ZY1hCT1NuZHhia28zWWpNdVUyVjVTakJKYW05cFdrY3dhVXhEU25wSmFuQTNTVzVXZVdGVFNUWkpibVI2VDJrNGRtSkhPV3BaVjNodllqTk9NRTlxWnpOT2FsbHBURU5LYUVscWNHSkpiVkp3V2tkT2RtSlhNSFprYWtscFdGTjNhV05wU1RaWE1URTVabEVqYTJWNUxUSSIsICJhcHYiOiAiN1B2cUNRS0pvbDQwN3pwLW5jeHBKSTZvZmNoQkxBeGJFVVhYSl9COHJEdyIsICJlcGsiOiB7ImNydiI6ICJYMjU1MTkiLCAia3R5IjogIk9LUCIsICJ4IjogIm5ESUQyaUQzUTNjNDBVOVV1VVlLZUh3aWZ0aC1yenY2NjFPeERYNTVRMGsifSwgInNraWQiOiAiZGlkOnBlZXI6Mi5WejZNa3NraXB5b28yR0tyemFIaENMdkpGVDZZY1VRcUw1ZzM3WVRNam1YSExYV3V3LkV6NkxTbkVIWFo3OVlCQmRjWXRDOXhFRmo1dVIySmhlajEzMjNYcXBOSndxbko3YjMuU2V5SjBJam9pWkcwaUxDSnpJanA3SW5WeWFTSTZJbmR6T2k4dmJHOWpZV3hvYjNOME9qZzNOallpTENKaElqcGJJbVJwWkdOdm

# Process inbox for Subscription Acknowledgement

In [17]:
inbox

[{'type': 'https://didcomm.org/musig2/subscribe/accept',
  'id': '6ba57cc3-55e2-47fd-b5e4-42189ce85857',
  'body': {},
  'from': 'did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ',
  'to': 'did:peer:2.Vz6Mkskipyoo2GKrzaHhCLvJFT6YcUQqL5g37YTMjmXHLXWuw.Ez6LSnEHXZ79YBBdcYtC9xEFj5uR2Jhej1323XqpNJwqnJ7b3.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjYiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ'}]

In [18]:
musig2_coordinators = []


In [19]:
def process_inbox(inbox):
    while len(inbox) != 0:
        message = inbox.pop(0)
        if message["type"] == "https://didcomm.org/musig2/subscribe/accept":
            coordinator = message["from"]
            if (coordinator == coordinator_did):
                print(f"Subscribed to a MuSig2 coordinator {coordinator}")
                musig2_coordinators.append(coordinator)
            else:
                print(coordinator)
                print(coordinator_did)
                

In [20]:
process_inbox(inbox)

Subscribed to a MuSig2 coordinator did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ


In [21]:
musig2_coordinators

['did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ']

In [22]:
inbox

[{'type': 'https://didcomm.org/musig2/keygen/new_cohort',
  'id': 'c005a99f-093e-4597-ab5c-dee23fe174e3',
  'thread_id': '5d51fed4-1e48-4da1-a495-0296dd73167e',
  'body': {},
  'from': 'did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ',
  'to': ['did:peer:2.Vz6Mkskipyoo2GKrzaHhCLvJFT6YcUQqL5g37YTMjmXHLXWuw.Ez6LSnEHXZ79YBBdcYtC9xEFj5uR2Jhej1323XqpNJwqnJ7b3.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjYiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ']}]

In [23]:
def create_join_cohort_message(to, thread_id, participant_pk):
    
    message = {
        "type": "https://didcomm.org/musig2/subscribe",
        "id": str(uuid.uuid4()),
        "thread_id": thread_id,
        "body": {
            "participant_pk": participant_pk
        },
        "from": did,
        "to": [to],
    }
    return message

In [1]:
from buidl.ecc import PrivateKey

from buidl.mnemonic import secure_mnemonic
from buidl.hd import HDPrivateKey

## Run this if you want a new hardware key
mnemonic = secure_mnemonic()

# mnemonic = "prosper can dial lumber write coconut express imitate husband isolate inside release brush media please kind comic pill science repeat basic also endorse bronze"
print("Mnemonic : ", mnemonic)

root_hdpriv = HDPrivateKey.from_mnemonic(mnemonic, network="signet")

purpose = "340"

initial_sk = root_hdpriv.get_private_key(purpose, address_num=2)
initial_pk = initial_sk.point


participant_pk = initial_pk.sec().hex()
print(f"Participant PK: {participant_pk}")


Mnemonic :  waste again network differ second draw same plug current trust there gold drift salad will olive develop shoe mammal board nature repeat bargain more
Participant PK: 028e26d03605c7b4a56efcfe5e261cca515a00178dd088d19a59187ccfc9f839cf


In [ ]:
for message in inbox:
    if message["type"] == "https://didcomm.org/musig2/keygen/new_cohort":
        frm = message["from"]
        thread_id = message["thread_id"]
        reply = create_join_cohort_message([frm],thread_id,participant_pk)
        print(reply)
        asyncio.create_task(send_message(reply, frm, did))

In [ ]:
inbox